## Get more word matrix 

In [1]:
import keras
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling,lecun_normal
from keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import seaborn as sns

Using TensorFlow backend.


In [5]:
from clustering_class import *
from combine_models import *
from utils import *
from DEC import *
from generate_word_vector import *

## Get different embadding matrix

In [6]:
decs=pd.read_csv('data/clean_desc_data.csv',index_col=0)
decs.rename(columns={'ticker':'company'},inplace=True)
f = open("data/baidu_stopwords.txt", "r")
stopwords = f.read().splitlines()
f.close()

In [7]:
## read bert matrix
cleaned_dat = pd.read_csv("data/clean_desc_data.csv")
companylist = cleaned_dat["ticker"]
short = cleaned_dat[cleaned_dat["intro"].str.len() <= 512]
big = cleaned_dat[cleaned_dat["intro"].str.len() > 512]
short_intro = short["intro"].values.tolist()
big_first = pd.concat([big["ticker"], big["intro"].str[:512]], axis=1)
big_second = pd.concat([big["ticker"], big["intro"].str[512:1024]], axis=1)
big_second = big_second[big_second.iloc[:, 1].str.len() > 100]
short_intro = short["intro"].values.tolist()
big_first_intro = big_first["intro"].values.tolist()
big_second_intro = big_second["intro"].values.tolist()

big_bert1 = pd.read_csv("bert_vec_output/first_long_CLS_long_bert.csv").iloc[:, 1:]
big_bert2 = pd.read_csv("bert_vec_output/second_long_CLS_long_bert.csv").iloc[:, 1:]
short_bert = pd.read_csv("bert_vec_output/short_CLS_long_bert.csv").iloc[:, 1:]
all_data = {}
filte = []
for i in big_first["ticker"]:
    filte.append(True) if i in big_second["ticker"].values else filte.append(False)
for i in range(6):
    big_bert1_i = big_bert1.iloc[:, i * 1024 : (i + 1) * 1024].copy()
    big_bert2_i = big_bert2.iloc[:, i * 1024 : (i + 1) * 1024].copy()
    short_bert_i = short_bert.iloc[:, i * 1024 : (i + 1) * 1024].copy()
    big_bert2_i.index = big_bert1_i[filte].index
    big_bert1_i[filte] = (big_bert1_i[filte] + big_bert2_i) / 2
    big.index = big_bert1_i.index
    short.index = short_bert_i.index
    big_bert1_i.insert(loc=0, column="company", value=big["ticker"])
    short_bert_i.insert(loc=0, column="company", value=short["ticker"])
    all_data['the %d bert'%i]=pd.concat([short_bert_i, big_bert1_i], axis=0,ignore_index=True)

In [8]:
company_list=decs['company'].values
desc_list=decs['intro'].values
vector_generator=generate_doc_vector(company_list,desc_list,stopwords,0.8,5)
bag_of_words=vector_generator.bag_of_words()
all_data['bag of words']=bag_of_words

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.521 seconds.
Prefix dict has been built successfully.


In [9]:
for i in range(50,401,50):
    doc2vec_1=vector_generator.doc2vector(vector_size=i,alpha=0.01)
    doc2vec_2=vector_generator.doc2vector(vector_size=i,alpha=0.01,dm=0)
    all_data['pvdm %d length'%i]=doc2vec_1
    all_data['pvdbow %d length'%i]=doc2vec_2

In [10]:
for i in range(200,1101,100):
    Lsi=vector_generator.LSI(num_topics=i)
    all_data['lsi %d length'%i]=Lsi

In [14]:
len(set(pd.read_csv('data/official_class.csv').official_class))

24

In [15]:
print(list(zip(range(24),list(all_data.keys()))))

[(0, 'the 0 bert'), (1, 'the 1 bert'), (2, 'the 2 bert'), (3, 'the 3 bert'), (4, 'the 4 bert'), (5, 'the 5 bert'), (6, 'bag of words'), (7, 'pvdm 50 length'), (8, 'pvdbow 50 length'), (9, 'pvdm 100 length'), (10, 'pvdbow 100 length'), (11, 'pvdm 150 length'), (12, 'pvdbow 150 length'), (13, 'pvdm 200 length'), (14, 'pvdbow 200 length'), (15, 'pvdm 250 length'), (16, 'pvdbow 250 length'), (17, 'pvdm 300 length'), (18, 'pvdbow 300 length'), (19, 'pvdm 350 length'), (20, 'pvdbow 350 length'), (21, 'pvdm 400 length'), (22, 'pvdbow 400 length'), (23, 'lsi 200 length')]


In [12]:
import pickle
with open('all_vec_dat','wb') as f:
    pickle.dump(all_data,f)
    f.close()

## Generate Classifier

In [ ]:
import pickle
with open('all_vec_dat','rb') as f:
    all_data=pickle.load(f)
    f.close()

In [21]:
tol = 0.001
maxiter = 1000
update_interval = 100
n_clusters = 26
n_epochs=200
batch_size=128
init = lecun_normal()
pretrain_optimizer = SGD(lr=1, momentum=0.9)
optimizer=SGD(lr=0.01,momentum=0.9,decay=0.01/n_epochs)
output=[]
kmeans = KMeans(n_clusters=n_clusters)

In [ ]:
def scheduler(epoch):
    if epoch %10== 0:
        model.lr.set_value(.02)
    return model.lr.get_value()

change_lr = LearningRateScheduler(scheduler)


In [22]:
for key,input in all_data.items():
    input_x=input.iloc[:,1:]
    dims=[input_x.shape[-1],500,200,2000,50]
    autoencode, encoder = autoencoder(dims, act='relu', init=init, dropout=0.2)
    autoencode.compile(optimizer=pretrain_optimizer, loss='mse')
    autoencode.fit(input_x, input_x, batch_size=batch_size, epochs=n_epochs)
    kmeans=kmeans.fit(encoder.predict(input_x))
    y_init=kmeans.cluster_centers_
    loss = 0
    index = 0
    index_array = np.arange(input_x.shape[0])
    clustering_layer = ClusteringLayer(n_clusters=n_clusters, name='clustering',weights=[y_init])(encoder.output)
    model = Model(inputs=encoder.input, outputs=clustering_layer)    
    model.compile(loss='kld', optimizer=optimizer)
    y_pred_last=kmeans.predict(encoder.predict(input_x))
    for ite in range(maxiter):
        if ite % update_interval == 0:
            q = model.predict(input_x, verbose=0)
            p = target_distribution(q)  # update the auxiliary target distribution p

            # evaluate the clustering performance
            y_pred = q.argmax(1)

            # check stop criterion
            delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
            y_pred_last = np.copy(y_pred)
            print('the '+str(ite)+'th training on '+key)
            if ite > 0 and delta_label < tol:
                print('delta_label = ', delta_label, ' < %d'%tol)
                print('Reached tolerance threshold. Stopping training.')
                break
        idx = index_array[index * batch_size: min((index+1) * batch_size, input_x.shape[0])]
        loss = model.train_on_batch(x=input_x.iloc[idx], y=p[idx])
        index = index + 1 if (index + 1) * batch_size <= input_x.shape[0] else 0
    q = model.predict(input_x,verbose=0)
    clas=pd.DataFrame(np.argmax(q,1),columns=[key])
    clas.insert(0,'company',input['company'])
    output.append(clas)


Epoch 1/200
3924/3924 [==============================] - 1s 345us/step - loss: 0.1606
Epoch 2/200
3924/3924 [==============================] - 1s 191us/step - loss: 0.0366
Epoch 3/200
3924/3924 [==============================] - 1s 200us/step - loss: 0.0237
Epoch 4/200
3924/3924 [==============================] - 1s 195us/step - loss: 0.0232
Epoch 5/200
3924/3924 [==============================] - 1s 196us/step - loss: 0.0177
Epoch 6/200
3924/3924 [==============================] - 1s 193us/step - loss: 0.0165
Epoch 7/200
3924/3924 [==============================] - 1s 192us/step - loss: 0.0163
Epoch 8/200
3924/3924 [==============================] - 1s 195us/step - loss: 0.0158
Epoch 9/200
3924/3924 [==============================] - 1s 192us/step - loss: 0.0154
Epoch 10/200
3924/3924 [==============================] - 1s 191us/step - loss: 0.0155
Epoch 11/200
3924/3924 [==============================] - 1s 192us/step - loss: 0.0148
Epoch 12/200
3924/3924 [============================

In [35]:
from sklearn.preprocessing import LabelEncoder
official_df=pd.read_csv('data/official_class.csv',index_col=0)
le=LabelEncoder()
le_class=le.fit_transform(official_df['official_class'])
chinese_class=official_df['official_class']
official_df['official_class']=le_class
print('there are %d industries in official classification'%len(set(chinese_class)))
print('there are %d companys in official classification'%len(chinese_class))
official_df.head()

there are 24 industries in official classification
there are 3802 companys in official classification


,company,official_class
0,000001.SZ,21
1,000002.SZ,11
2,000004.SZ,17
3,000005.SZ,18
4,000006.SZ,11


In [40]:
combiner=combine_models()
all_df=[official_df]+output
all_cluster=combiner.get_combine_cluster(all_df)
all_cluster.head()

,company,official_class,the 0 bert,the 1 bert,the 2 bert,the 3 bert,the 4 bert,the 5 bert,bag of words,pvdm 50 length,...,lsi 200 length,lsi 300 length,lsi 400 length,lsi 500 length,lsi 600 length,lsi 700 length,lsi 800 length,lsi 900 length,lsi 1000 length,lsi 1100 length
0,000001.SZ,21,5,9,8,9,24,4,18,14,...,14,12,22,2,23,1,0,22,9,8
1,000002.SZ,11,5,9,8,9,5,18,12,9,...,8,20,22,7,3,17,15,22,6,5
2,000004.SZ,17,0,13,20,1,4,17,11,2,...,8,20,22,20,3,17,15,22,17,10
3,000005.SZ,18,9,11,6,9,9,21,4,24,...,8,20,22,21,3,17,10,22,7,10
4,000006.SZ,11,5,9,8,9,5,18,17,21,...,8,20,22,7,3,16,15,22,7,8


In [33]:
all_cluster.to_csv('all_cluster.csv')

In [29]:
price = pd.read_csv("data/full stock price 20190630-20191231.csv", index_col=0)
price["date"] = pd.to_datetime(price["date"], format="%Y-%m-%d")
price = trans_ticker(price)
price.sort_values(by=['symbol','date'],inplace=True)
stock_return=price.groupby(by=['symbol'])['adjusted'].pct_change()
price['adjusted']=stock_return
price.dropna(inplace=True)
price.set_index(pd.Index(range(len(price))),inplace=True)
price.rename(columns={'symbol':'company','adjusted':'return'},inplace=True)
price.head()

,company,date,return
0,000001.SZ,2019-07-02,0.017947
1,000001.SZ,2019-07-03,-0.011989
2,000001.SZ,2019-07-04,-0.001428
3,000001.SZ,2019-07-05,-0.005004
4,000001.SZ,2019-07-08,-0.023707


In [41]:
second_analysis=performance_analysis(mode='return_performance',class_df=all_cluster,return_df=price)

In [42]:
second_analysis.get_wellness()

/tf/industry-classification/combine_models.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.return_map[name][cl].dropna(1,'all',inplace=True)
/tf/industry-classification/combine_models.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.return_map[name][cl]['return']=self.return_map[name][cl].mean(1)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/usr/local/lib/python3.6/dist-packages/scipy/stats/_stats_mstats_common.py:142: Runtime

In [50]:
for name,result in second_analysis.R2:
    print('Mean of R square of {} classification is: {}'.format(name, result))

Mean of R square of official_class classification is: 0.37561767015650677
Mean of R square of the 0 bert classification is: 0.3353138304454243
Mean of R square of the 1 bert classification is: 0.34251585488141423
Mean of R square of the 2 bert classification is: 0.32881035315609625
Mean of R square of the 3 bert classification is: 0.33422180655666045
Mean of R square of the 4 bert classification is: 0.31851686554375985
Mean of R square of the 5 bert classification is: 0.33299801598802725
Mean of R square of bag of words classification is: 0.41388696992050394
Mean of R square of pvdm 50 length classification is: 0.3628641545405551
Mean of R square of pvdbow 50 length classification is: 0.4001652149846745
Mean of R square of pvdm 100 length classification is: 0.343200449241978
Mean of R square of pvdbow 100 length classification is: 0.3595508620607603
Mean of R square of pvdm 150 length classification is: 0.3767013426512649
Mean of R square of pvdbow 150 length classification is: 0.41388